In [5]:
import pyodbc
import psycopg2
from abc import ABC, abstractmethod

class DataSource(ABC):
    def __init__(self, connection_details):
        self.connection_details = connection_details
    
    @abstractmethod
    def test_connection(self):
        pass

    @abstractmethod
    def get_schemas(self):
        pass

    @abstractmethod
    def get_tables(self, schema):
        pass

    @abstractmethod
    def get_table_metadata(self, schema, table):
        pass

class AzureSQLDatabase(DataSource):
    def __init__(self, connection_details):
        super().__init__(connection_details)
        self.connection = None

    def connect(self):
        if not self.connection:
            self.connection = pyodbc.connect(self.connection_details)

    def test_connection(self):
        try:
            self.connect()
            cursor = self.connection.cursor()
            cursor.execute("SELECT 1")
            cursor.fetchone()
            return True
        except Exception as e:
            print(f"Connection test failed: {e}")
            return False

    def get_schemas(self):
        self.connect()
        cursor = self.connection.cursor()
        cursor.execute("SELECT schema_name FROM information_schema.schemata")
        return [row[0] for row in cursor.fetchall()]

    def get_tables(self, schema):
        self.connect()
        cursor = self.connection.cursor()
        cursor.execute(f"SELECT table_name FROM information_schema.tables WHERE table_schema = '{schema}'")
        return [row[0] for row in cursor.fetchall()]

    def get_table_metadata(self, schema, table):
        self.connect()
        cursor = self.connection.cursor()
        cursor.execute(f"""
            SELECT column_name, data_type
            FROM information_schema.columns
            WHERE table_schema = '{schema}' AND table_name = '{table}'
        """)
        return {row[0]: row[1] for row in cursor.fetchall()}

class PostgreSQLDatabase(DataSource):
    def __init__(self, connection_details):
        super().__init__(connection_details)
        self.connection = None

    def connect(self):
        if not self.connection:
            self.connection = psycopg2.connect(self.connection_details)

    def test_connection(self):
        try:
            self.connect()
            cursor = self.connection.cursor()
            cursor.execute("SELECT 1")
            cursor.fetchone()
            return True
        except Exception as e:
            print(f"Connection test failed: {e}")
            return False

    def get_schemas(self):
        self.connect()
        cursor = self.connection.cursor()
        cursor.execute("SELECT schema_name FROM information_schema.schemata")
        return [row[0] for row in cursor.fetchall()]

    def get_tables(self, schema):
        self.connect()
        cursor = self.connection.cursor()
        cursor.execute(f"SELECT table_name FROM information_schema.tables WHERE table_schema = '{schema}'")
        return [row[0] for row in cursor.fetchall()]

    def get_table_metadata(self, schema, table):
        self.connect()
        cursor = self.connection.cursor()
        cursor.execute(f"""
            SELECT column_name, data_type
            FROM information_schema.columns
            WHERE table_schema = '{schema}' AND table_name = '{table}'
        """)
        return {row[0]: row[1] for row in cursor.fetchall()}

# Mock function for retrieving secrets from Azure Key Vault
def get_secret_from_key_vault(secret_name):
    # This function should retrieve the secret from Azure Key Vault.
    # Here, it's mocked for the sake of example.
    secrets = {
        'azure_sql_connection': 'DRIVER={ODBC Driver 17 for SQL Server};SERVER=server_name;DATABASE=database_name;UID=user;PWD=password',
        'postgresql_connection': 'dbname=database_name user=user password=password host=host port=port'
    }

    #call the functions to get the secret keys from the vault
    return secrets.get(secret_name)

# Example usage
if __name__ == "__main__":
    azure_sql_details = get_secret_from_key_vault('azure_sql_connection')
    postgres_details = get_secret_from_key_vault('postgresql_connection')

    azure_sql_db = AzureSQLDatabase(azure_sql_details)
    postgres_db = PostgreSQLDatabase(postgres_details)

    print("Azure SQL Database:")
    if azure_sql_db.test_connection():
        print("Schemas:", azure_sql_db.get_schemas())
        print("Tables in schema 'dbo':", azure_sql_db.get_tables('dbo'))
        print("Metadata for table 'your_table':", azure_sql_db.get_table_metadata('dbo', 'your_table'))

    print("\nPostgreSQL Database:")
    if postgres_db.test_connection():
        print("Schemas:", postgres_db.get_schemas())
        print("Tables in schema 'public':", postgres_db.get_tables('public'))
        print("Metadata for table 'your_table':", postgres_db.get_table_metadata('public', 'your_table'))


Azure SQL Database:
Connection test failed: ('IM002', '[IM002] [Microsoft][ODBC Driver Manager] Data source name not found and no default driver specified (0) (SQLDriverConnect)')

PostgreSQL Database:
Connection test failed: invalid integer value "port" for connection option "port"



In [4]:
!pip install psycopg2

   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   - -------------------------------------- 0.0/1.2 MB 660.6 kB/s eta 0:00:02
   --------- ------------------------------ 0.3/1.2 MB 3.0 MB/s eta 0:00:01
   ---------------------------------------- 1.2/1.2 MB 8.2 MB/s eta 0:00:00
